# ISE-533 Project 4: Allocation Problem for COVID-19 Ventilators
## Point Forecast - Deterministic Model

By: Jacob Andreesen, Jeff Chen, Miao Xu, Yiyi Wang

In [ ]:
import os
import numpy as np
import pandas as pd
import math
from pyomo.environ import *
from collections import defaultdict
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from datetime import timedelta

In [ ]:
raw = pd.read_csv('project 4 dataset/init_data.csv', header=0, index_col=0)
# raw.loc[51, 'StateCode'] = 'SNS'
# raw.loc[51, 'Available capacity'] = 12000
# raw.loc[51, 'nb_idx'] = str([i for i in range(51)])
raw

In [ ]:
first_date = '2020/03/25'
t = 7
T = 14
P = 10
lbd = 0.1

S = 51
states_name = raw['State']
SNS_stock = 12000

init_ratio = 0.8
flow_bound_ratio = 0.3 #0.2
stock_bound_ratio = 0.5 #0.6

In [ ]:
#first iteration values
data = raw.rename(columns={'Available capacity': "stock_self"})
data = data[:-1]
data['stock_self'] = init_ratio * data['stock_self']
data['stock_nb'] = None
for i in range(S):
    data.loc[i, 'stock_nb'] = str([0 for _ in eval(data.loc[i, 'nb'])])
data['stock_sns'] = [0 for _ in range(S)]
data

In [ ]:
idx = 0
flow_mapping = {}
flow_mapping_rev = {}
out_flow = defaultdict(list)
in_flow = defaultdict(list)

for i in range(S):
    nbs = eval(data.iloc[i, 3])
    out_flow[i] = nbs
    for n in nbs:
        flow_mapping[(i, n)] = idx
        flow_mapping_rev[idx] = (i, n)
        in_flow[n].append(i)
        idx += 1

in_flow[51] = []
out_flow[51] = list(range(51))
for n in range(S):
    flow_mapping[(51, n)] = idx
    flow_mapping_rev[idx] = (51, n)
    idx += 1

In [ ]:
model = ConcreteModel()

#Sets
F = idx
model.time_set = list(range(1, t + 1))
model.state_set = list(range(S))
model.state_p_set = list(range(S + 1))
model.flow_set = list(range(F))
model.flow_no_sns_set = list(range(F - S))

In [ ]:
# instantiate Param
ini_self = [round(i) for i in data['stock_self']]
ini_nb = defaultdict(list)
for i in range(S):
    ini_nb[i] = eval(data.loc[i, 'stock_nb'])
ini_SNS = [round(i) for i in data['stock_sns']]

# Parameters
model.demand = Param(model.state_set, model.time_set, within=NonNegativeReals, mutable=True)

model.U = [round(flow_bound_ratio * ini_self[flow_mapping_rev[j][0]]) for j in model.flow_no_sns_set] + [round(flow_bound_ratio * init_ratio * SNS_stock) for _ in range(S)]
model.G = [round(stock_bound_ratio * i) for i in ini_self]

In [ ]:
# Var
model.s0 = Var(model.state_p_set, within=NonNegativeReals)
model.s0_nb = Var(model.flow_set, within=NonNegativeReals)
model.x = Var(model.flow_set, within=NonNegativeReals)
model.delta = Var(model.state_set, model.time_set, within=NonNegativeReals)

In [ ]:
#Constraints
model.stock_self = Constraint(model.state_set, rule=lambda model, j: model.s0[j] + sum(model.x[flow_mapping[(j, i)]] for i in out_flow[j]) == ini_self[j])
model.stock_SNS_self = Constraint(rule=lambda model: model.s0[51] + sum(model.x[flow_mapping[(51, i)]] for i in out_flow[51]) == init_ratio * SNS_stock)

model.stock_nb = ConstraintList()
for j in model.state_set:
    for dum, i in enumerate(out_flow[j]):
        f = flow_mapping[(i, j)] 
        model.stock_nb.add(model.s0_nb[f] - model.x[f] == ini_nb[j][dum])
    f = flow_mapping[(51, j)] 
    model.stock_nb.add(model.s0_nb[f] - model.x[f] == ini_SNS[j])
        
model.delta_bound = ConstraintList()
for j in model.state_set:
    for tt in model.time_set:
        total_stock = model.s0[j] + model.s0_nb[flow_mapping[(51, j)] ] + sum(model.s0_nb[flow_mapping[(i, j)]] for i in out_flow[j])
        model.delta_bound.add(model.delta[j, tt] + total_stock >= model.demand[j, tt])
        

model.s0_bound = Constraint(model.state_set, \
                                  rule=lambda model, j: model.s0[j] >= model.G[j])
model.x_bound = Constraint(model.flow_set, \
                                  rule=lambda model, j: model.x[j] <= model.U[j])

In [ ]:
#Objective
model.unmet = Expression(initialize=(sum(model.delta[s, tt] for s in model.state_set for tt in model.time_set)))
model.penalty = Expression(initialize=(sum(lbd*model.s0_nb[i] for i in model.flow_no_sns_set)))
model.obj = Objective(expr = model.unmet + model.penalty, sense = minimize)

### Solve Allocation Problem for 10 Weeks

In [ ]:
for i in range(10):
    date = pd.to_datetime(first_date) + timedelta(7*i)
    print("date:", date)
    for i in range(S):
        state = data.loc[i, 'State']
        df = pd.read_csv('project 4 dataset/prediction/{}/{}.csv'.format(date.strftime('%m%d'), state), header=0, index_col=0)
        for tt in range(t):
            model.demand[i, tt + 1].value = df.iloc[tt, 1]
    
    SolverFactory('glpk').solve(model) #.write()

    data['stock_all'] = [0 for _ in range(S)] 
    for i in model.state_set:
        data.loc[i, 'stock_self'] = round(model.s0[i].value)
        data.loc[i, 'stock_all'] += data.loc[i, 'stock_self']

    for i in model.state_set:
        l = eval(data.loc[i, 'nb_idx'])
        new_nb = []
        for n in l:
            j = flow_mapping[(n, i)]
            new_nb.append(round(model.s0_nb[j].value))
        data.loc[i, 'stock_nb'] = str(new_nb)
        data.loc[i, 'stock_all'] += sum(new_nb)

    for i in model.state_set:
        j = flow_mapping[(51, i)]
        data.loc[i, 'stock_sns'] = round(model.s0_nb[j].value)
        data.loc[i, 'stock_all'] += data.loc[i, 'stock_sns']
        
    data.to_csv(date.strftime('%m%d') + '.csv')

### Compare results with actual demand

In [ ]:
states_name = data['State']
state_demand = pd.DataFrame(columns=['date']) 

for j in range(S):
    state = states_name[j]
    temp = pd.read_csv("project 4 dataset/eval/{}.csv".format(state), header=None)
    temp.rename(columns = {0: 'date', 1: state}, inplace = True)
    state_demand = state_demand.merge(temp, on='date', how='outer') 

state_demand = state_demand.set_index('date')
state_demand = state_demand.reset_index()
state_demand

In [ ]:
data_list = [states_name]

for i in range(10):
    date = pd.to_datetime(first_date) + timedelta(7*i)
    stock = pd.read_csv("{}.csv".format(date.strftime('%m%d')))
    #stock = stock.drop(['Unnamed: 0'], axis = 1)
    data_list.append(stock['stock_all'])
    
final = pd.concat(data_list, axis = 1)
#stock
final

In [ ]:
eval = state_demand.groupby(np.arange(len(state_demand))//7).mean()

final1 = final.transpose()
final1 = final1.iloc[1:]
final1.index = eval.index
final1.columns = states_name
final1

In [ ]:
final_repeat = pd.DataFrame()
final_repeat = final1.loc[final1.index.repeat(7)].reset_index(drop=True)
final_repeat

In [ ]:
unmet_everyday = state_demand.drop(['date'], axis = 1).subtract(final_repeat)
# unmet_everyday
unmet_everyday[unmet_everyday <= 0] = 0
unmet_everyweek = unmet_everyday.groupby(np.arange(len(unmet_everyday))//7).sum()

unmet_everyweek

In [ ]:
result = pd.DataFrame(unmet_everyweek.sum(axis = 1), columns = ['Unmet Demand'])
result.index = [1,2,3,4,5,6,7,8,9,10]
result = result.rename_axis('Week').reset_index()
#result.to_csv('project 4 dataset/result_det.csv')
result

In [ ]:
pd.DataFrame(result.sum(axis = 1), columns = ['Unmet Demand']).sum(axis = 0)